In [24]:
#My path
path = '/Users/mhalt/Desktop/DEEPLEARNINGPROJECT/Data/'

#Your path
#path = '/Users/claes/Desktop/Deep_learning/Week_6'

## Lægge alle data fra hver fryser sammen, klar til autoencoder

In [ ]:
import pandas as pd
import glob
from sklearn.preprocessing import MinMaxScaler

# Step 1: Load all parquet files matching the pattern
all_files = glob.glob(path + "cleaned_data_*.parquet")

# Step 2: Define the columns to drop (conditionally)
drop_columns_set_1 = ['Datetime', 'State', 'Type', 'Event']
drop_columns_set_2 = drop_columns_set_1 + ['main_fault']

# Step 3: Initialize a list to store dataframes
dfs = []

# Step 4: Load, preprocess, and concatenate data
for file in all_files:
    df = pd.read_parquet(file)
    
    # Drop columns that exist in the dataframe
    if 'main_fault' in df.columns:
        df.drop(columns=[col for col in drop_columns_set_2 if col in df.columns], inplace=True)
    else:
        df.drop(columns=[col for col in drop_columns_set_1 if col in df.columns], inplace=True)
    
    # Append the processed dataframe to the list
    dfs.append(df)

# Step 5: Combine all dataframes into one
combined_df = pd.concat(dfs, axis=0, ignore_index=True)

# Step 6: Normalize the combined dataset
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(combined_df)

# Convert back to a DataFrame for easier handling (optional)
normalized_df = pd.DataFrame(data_normalized, columns=combined_df.columns)

normalized_df.to_parquet(path + 'combined_cleaned_data.parquet', index=False)


In [26]:
print(f"Længde af combined df: {normalized_df.shape[0]:,}")

Længde af combined df: 2,923,224
